In [1]:
import importlib
import warnings
from logging import getLogger
from types import ModuleType
from process_bigraph import ProcessTypes
from bigraph_schema.protocols import function_module
from unum import Unum
from importlib import import_module
from typing import *

from wholecell.utils import units
from ecoli.experiments.ecoli_master_sim import EcoliSim
from ecoli.shared import ecoli_core as ec
from ecoli.shared.types.register import test_register_type
from ecoli.shared.vivarium_instances.factory import VivariumFactory
from ecoli.shared.utils.schemas import listener_schema, get_defaults_schema, collapse_defaults
vivarium = VivariumFactory()

from ecoli.shared.registry import ModelProcesses
import json 



replication = {
    "x": {**listener_schema({
        "fork_coordinates": [],
        "fork_domains": [],
        "fork_unique_index": [],
        "number_of_oric": [],
        "free_DnaA_boxes": [],
        "total_DnaA_boxes": [],
    })},
}
    


replication
get_defaults_schema(replication)

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


{'x': {'fork_coordinates': list,
  'fork_domains': list,
  'fork_unique_index': list,
  'number_of_oric': list,
  'free_DnaA_boxes': list,
  'total_DnaA_boxes': list}}

In [2]:
from ecoli.migrated.listeners.replication_data import ReplicationData


proc = ReplicationData(core=ec)

s = collapse_defaults(proc.input_schema)
proc.update(s, 11)

ValueError: no field of name coordinates

In [24]:
import numpy as np


bulk_dtype: np.dtype = np.dtype([
    ("id", "<U100"),
    ("count", "<f8")
])

np.array([('a', 11)], dtype=bulk_dtype).dtype

dtype([('id', '<U100'), ('count', '<f8')])

In [ ]:
configpath = '/Users/alexanderpatrie/Desktop/repos/ecoli/v2Ecoli/ecoli/composites/ecoli_configs/test_installation.json'
sim = EcoliSim.from_file(configpath)

In [ ]:
import numpy as np 


doc = {
    'complexation': {
        '_type': 'process',
        'address': 'local:complexation',
        'config': {
            "stoichiometry": [[-1, 1, 0], [0, -1, 1], [1, 0, -1], [-1, 0, 1], [1, -1, 0], [0, 1, -1]],
            "rates": np.random.random((6,)).tolist(),
            "molecule_names": ["A", "B", "C"],
            "seed": 1,
            "reaction_ids": [1, 2, 3, 4, 5, 6],
            "complex_ids": [1, 2, 3, 4, 5, 6],
        },
        'inputs': {
            'bulk': ['bulk'],
            'timestep': ['timestep']
        }
    }
}

In [ ]:
# flow represents which processes should be connected
# topology represents the overall ports for the composite(emitter?)
# processes represents the actual processes involved.

# THUS, to create the composite doc we should:
# 1. For each key(process) in sim.processes, create the composite doc as usual:
# 1a. 


def construct_state(sim: EcoliSim):
    sim_processes = sim.processes
    state = {}
    for process_id in sim_processes:
        state[process_id] = {
            "address": f"local:{process_id}",
            "config": sim.process_configs.get(process_id),
            "inputs": {}
        }
